In [12]:
# Useful starting lines
%matplotlib inline
import numpy as np
import datetime
import matplotlib.pyplot as plt

from helpers import *
from implementations import *
from Data_Cleaning import *
from training_procedure import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
y, tx, ids = load_csv_data("train.csv", sub_sample=False)

In [14]:
# force meaningless data into the means
tx_mean_meaningful = np.zeros([tx.shape[1],])
for i in range(1, tx.shape[1]):
    tx_mean_meaningful[i] = np.mean(tx[:, i][tx[:, i] != -999])
    tx[:, i][tx[:, i] == -999] = tx_mean_meaningful[i]

# delete categorical column '22'
# tx = np.delete(tx, 22, axis = 1)

In [15]:
tx.shape

(250000, 30)

In [16]:
def replaceFirstFeature(tx, initial_gamma):
    correct = tx[tx[:,0]!=-999][:,1:]
    ncorrect = tx[tx[:,0]==-999][:, 1:]

    # # standardization
    # correct = standardize(correct)
    # ncorrect = standardize(ncorrect)

    num_feat = correct.shape[1]
    # add cross quadratic terms
    for i in range(num_feat):
        new_features_xx = np.multiply(correct[:, i:num_feat], correct[:, 0:num_feat-i])
        new_features_xxN = np.multiply(ncorrect[:, i:num_feat], ncorrect[:, 0:num_feat-i])
        xx = np.hstack([correct, new_features_xx])
        xxN = np.hstack([ncorrect, new_features_xxN])

    # standardization
    xx = standardize(xx)
    xxN = standardize(xxN)

    # add the bias term 
    bias_term_xx = np.ones([xx.shape[0], 1])
    bias_term_xxN = np.ones([xxN.shape[0], 1])
    xx = np.hstack([bias_term_xx, xx])
    xxN = np.hstack([bias_term_xxN, xxN])

    w_init = np.zeros(xx.shape[1])
    best_w, best_loss = train_first_feature(tx[:,0][tx[:,0] != -999], xx, w_init, initial_gamma, 20, 0)
    newFirst = xxN.dot(best_w)
    print(f"Best loss: {best_loss}")
    tx[:,0][tx[:,0]==-999] = newFirst
    # tx[tx[:,0]==-999][:,0] = newFirst

    return tx

In [17]:
initial_gamma = 0.001
tx = replaceFirstFeature(tx, initial_gamma)

  5%|▌         | 1/20 [00:01<00:24,  1.32s/it]

Linear Regression SGD(0/19): loss=156.06728948788526


 10%|█         | 2/20 [00:02<00:23,  1.31s/it]

Linear Regression SGD(1/19): loss=155.80362675009775


 15%|█▌        | 3/20 [00:03<00:22,  1.32s/it]

...

Linear Regression SGD(17/19): loss=150.69614674329944


 95%|█████████▌| 19/20 [00:24<00:01,  1.31s/it]

Linear Regression SGD(18/19): loss=150.7767325831206


100%|██████████| 20/20 [00:26<00:00,  1.31s/it]

Linear Regression SGD(19/19): loss=150.71852435061675
Best loss: 150.69614674329944


In [18]:
tx[tx[:,0]==-999][:,0], tx.shape

(array([], dtype=float64), (250000, 30))

In [19]:
model_training = split_according_num_split_jet(y, tx, ids)

In [20]:
tx_0 = model_training['0']['data']
tx_1 = model_training['1']['data']
tx_2 = model_training['2']['data']
tx_0.shape[0] + tx_1.shape[0] + tx_2.shape[0] == tx.shape[0]

True

In [21]:
tx_mean_meaningful_0 = np.zeros([tx_0.shape[1],])
for i in range(tx_0.shape[1]):
    tx_mean_meaningful_0[i] = np.mean(tx_0[:, i][tx_0[:, i] != -999])
    tx_0[:, i][tx_0[:, i] == -999] = tx_mean_meaningful_0[i]

In [ ]:
tx_mean_meaningful_1 = np.zeros([tx_1.shape[1],])
for i in range(tx_1.shape[1]):
    tx_mean_meaningful_1[i] = np.mean(tx_1[:, i][tx_1[:, i] != -999])
    tx_1[:, i][tx_1[:, i] == -999] = tx_mean_meaningful_1[i]

In [ ]:
tx_mean_meaningful_2 = np.zeros([tx_2.shape[1],])
for i in range(tx_2.shape[1]):
    tx_mean_meaningful_2[i] = np.mean(tx_2[:, i][tx_2[:, i] != -999])
    tx_2[:, i][tx_2[:, i] == -999] = tx_mean_meaningful_2[i]